#### Data Transformation

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\amitt\\Documents\\ML-OPS\\text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# types of data to be ingested
@dataclass
class DataTransformationConfig:
    root_dir : Path
    data_path : Path
    tokenizer_name : Path

In [6]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [7]:
#  this goes into the configuration file under config folder
class ConfigurationManager:
    def __init__( self,config_filepath = CONFIG_PATH, params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:


        config = self.config.data_transformation

        create_directories([config.root_dir])


        data_transformation_config = DataTransformationConfig(

            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
        )

        return data_transformation_config



In [ ]:
import os
from src.textsummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

#### Data Transformation Components

In [11]:
class DataTransformation:

    def __init__(self,config: DataTransformationConfig):

        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_to_examples(self,example_batch):


        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
        # use as a target tokenizer
        with self.tokenizer.as_target_tokenizer():

            target_encodings = self.tokenizer(example_batch["summary"], max_length=128, truncation=True)

        return {
                "input_ids": input_encodings["input_ids"],
                "attention_mask": input_encodings["attention_mask"],
                "labels": target_encodings["input_ids"]
        }
    
    def convert(self):

        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_examples, batched=True)

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))




In [13]:
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()

data_transformation = DataTransformation(config=data_transformation_config)

data_transformation.convert()

[2025-02-14 19:56:00,408: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-14 19:56:00,410: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-14 19:56:00,411: INFO: common: created directory at: artifacts]
[2025-02-14 19:56:00,412: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\amitt\Documents\ML-OPS\text-summarization\myenv\lib\site-packages\transformers\tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 162081.48 examples/s]
